In [53]:
import numpy as np
import random
import operator 
import pandas as pd 
import matplotlib.pyplot as plt
import math
from numba import autojit

In [150]:
def routeFitness(route, dm):
    dist = 0
    for i in range(len(route)-1):
        dist += dm[route[i], route[i+1]]
    dist += dm[route[-1], route[0]]
    return 1/dist

In [68]:
def dist_matrix(df):
    list_of_distances = [] # just to make it simple, probably there are 1k better ways to do this
    for point in df.itertuples():
        a = point[1]
        b = point[2]
        for point_s in df.itertuples():
            c = point_s[1]
            d = point_s[2]
            dist = math.sqrt(((c-a)**2) + ((d-b)**2))
            list_of_distances.append(dist)
    dist_matrix = np.array(list_of_distances) # convertion to array
    dist_matrix = np.reshape(dist_matrix, (len(df), len(df))) # reshaping
    # dist_matrix.shape # check
    return dist_matrix

In [69]:
def initPopulation(popSize, cities):
    population = []
    [population.append(random.sample(cities, len(cities))) for i in range(0, popSize)]
#     for i in range(0, popSize):
#         population.append(createRoute(cityList))
    return population

In [158]:
def GenA(permutation, dm, popSize, elite_n, mutation_r, n_of_gen):
    pop = initPopulation(popSize, permutation)
    fitness = {}
    [fitness.update({i:routeFitness(pop[i], dm)}) for i in range(0,len(pop))]
    rankedRoutes = sorted(fitness.items(), key = operator.itemgetter(1), reverse = True)
    print("Initial distance: " + str(1 / rankedRoutes[0][1]))
    
    for k in range(0, n_of_gen):
        
        ##################
        # Ranking routes #
        ##################
        
        fitness = {}
        #print(k)
        #[fitness.update({i:routeFitness(pop[i], dm)}) for i in range(0,len(pop))]
        for i in range(0, len(pop)):
#         # fitnessResults[i] = Fitness(population[i], dm).routeFitness()
            fitness[i] = routeFitness(pop[i], dm)
        
        rankedRoutes = sorted(fitness.items(), key = operator.itemgetter(1), reverse = True)
        
        #############
        # selection #
        #############
        
        selectionR = []
        df = pd.DataFrame(np.array(rankedRoutes), columns=["Index","Fitness"])
        df['cum_sum'] = df.Fitness.cumsum()
        df['cum_perc'] = 100*df.cum_sum/df.Fitness.sum()
        [selectionR.append(rankedRoutes[i][0]) for i in range(0, elite_n)]
        
        for i in range(0, len(rankedRoutes) - elite_n):
            epsilon = 100*random.random()
            for i in range(0, len(rankedRoutes)):
                if epsilon <= df.iat[i,3]:
                    selectionR.append(rankedRoutes[i][0])
                    break
        
        #print(selectionR)
        
        ###############
        # mating pool #
        ###############
        
        mp = []
       # [mp.append(pop[selectionR[i]]) for i in range(0, len(selectionR))]
        for i in range(0, len(selectionR)):
            index = selectionR[i]
            mp.append(pop[index])


        #############
        # Crossover #
        #############
        
        children = []
        length = len(mp) - elite_n
        pool = random.sample(mp, len(mp))
    
        [children.append(mp[i]) for i in range(0, elite_n)]
        
        for i in range(0, length):
            child = []
            childP1 = []
            childP2 = []
    
            geneA = int(random.random() * len(pool[i]))
            geneB = int(random.random() * len(pool[i]))
            
            startGene = min(geneA, geneB)
            endGene = max(geneA, geneB)
            [childP1.append(pool[j]) for j in range(startGene, endGene)]
            childP2 = [item for item in pool[len(mp)-i-1] if item not in childP1]
            
            child = childP1 + childP2
            
            children.append(child)
        #print(children)    
        ############
        # mutation #
        ############
        
        mutatedPop = []
    
        for ind in range(0, len(children)):
            individual = children[ind]
            for swapped in range(len(individual)):
                if(random.random() < mutation_r):
                    swapWith = int(random.random() * len(individual))
            
                    city1 = individual[swapped]
                    city2 = individual[swapWith]
            
                    individual[swapped] = city2
                    individual[swapWith] = city1
                    #print(individual)
            mutatedPop.append(individual)
       
        #print(mutatedPop)
        
        ##############
        # update pop #
        ##############
       # print(mutatedPop)
        
        pop = mutatedPop
         
    
    
    fitness = {}
    [fitness.update({i:routeFitness(pop[i], dm)}) for i in range(0,len(pop))]
    rankedRoutes = sorted(fitness.items(), key = operator.itemgetter(1), reverse = True)  
    print("Final distance: " + str(1 / rankedRoutes[0][1]))
    bestRouteIndex = rankedRoutes[0][0]
    bestRoute = pop[bestRouteIndex]
    return bestRoute

In [159]:
GenA(permutation = cities, popSize = 100, dm = d_matrix, elite_n = 10, mutation_r = 0.05, n_of_gen = 50)

Initial distance: 26034.469267891127


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [7]:
df = pd.read_csv("berlin52.tsp", header=None, sep = " ", names = ['x', 'y'], index_col = 0)
df.head()
cities = []
cities = list(range(len(df)))
d_matrix = dist_matrix(df)